In [1]:
import sys
sys.path.append(r'../dotNet/libs')

import clr
clr.AddReference("Microsoft.ML.Probabilistic")
clr.AddReference("Microsoft.ML.Probabilistic.Compiler")
clr.AddReference("Microsoft.ML.Probabilistic.Learners")

from System import Array, Double, Console

In [2]:
from Microsoft.ML.Probabilistic.Models import Variable, VariableArray, Range, InferenceEngine

### Example that works

This is the implementation of https://dotnet.github.io/infer/userguide/a%20simple%20example.html

In [3]:
firstCoin = Variable.Bernoulli(Double(0.5))
secondCoin = Variable.Bernoulli(Double(0.5))
bothHeads = firstCoin.op_BitwiseAnd(firstCoin, secondCoin)

In [4]:
engine = InferenceEngine()
# This should give 'Bernoulli(0.25)' since probability
# that a coin toss will give heads for both coins equals to 0.5 * 0.5 == 0.25
Console.WriteLine(f'Probability distribution both coins are heads: {engine.Infer(bothHeads)}');
# Now we observe a coin toss with observed value for both heads == false
bothHeads.ObservedValue = False;  
# Probability that the first coin is head == 1/3, since
# there are only 3 events possible without both coins giving heads
Console.WriteLine(f'Probability distribution over firstCoin: {engine.Infer(firstCoin)}');

### Example that doesn't work (yet)

Trying to implement https://dotnet.github.io/infer/userguide/Running%20inference.html, but this gives a problem.

Unfortunately we cannot call a specific instantiation of generic functions in pythonnet (see https://github.com/pythonnet/pythonnet/issues/641).

In [5]:
mean = Variable.GaussianFromMeanAndVariance(0.0, 100.0)
precision = Variable.GammaFromShapeAndScale(1.0, 1.0)
data = Variable.Observed(Array[Double]([11.0, 5.0, 8.0, 9.0]))

`data` should be of type `VariableArray[System.Double]`
but we get `Variable[System.Double[]]` instead. Type resolution is ambiguous here.

In [6]:
print(data.GetType())

Microsoft.ML.Probabilistic.Models.Variable`1[System.Double[]]


This can be resolved by specifying the type resolution explicitly on the function. But apparently this is not possible (yet) in pythonnet. The following throws an exception.

It seems the problem is that pythonnet cannot resolve overloads for generic methods.

In [7]:
data = Variable.Constant.Overloads[Double](Array[Double]([11.0, 5.0, 8.0, 9.0]))

TypeError: No match found for signature